In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [3]:
from mppi import Parsers as P

# Tutorial of the PwParser class

This tutorial describes the usage of the class PwParser of mppi used to extract information for the XML output file data-file-schema produced
by pw.

## Parse of a output file of Silicon 

__Follow the tutorial for the QeCalculator to produce the xml file used by the PwParser.__

The class is initialized by specifying the name of the xml file including its relative path 

In [4]:
results = P.PwParser('QeCalculator_test/si_scf_test1.save/data-file-schema.xml')

Parse file : QeCalculator_test/si_scf_test1.save/data-file-schema.xml


When the object is initialized several attributes are set, for instance

In [5]:
results.units

'Hartree atomic units'

In [6]:
print(results.natoms)
print(results.atomic_species)
print(results.atomic_positions)
print(results.nbands)
print(results.nkpoints)
print(results.spin_degen)

2
{'Si': ['2.808600000000000e1', 'Si.pbe-mt_fhi.UPF']}
[['Si', [-1.2875, 1.2875, 1.2875]], ['Si', [1.2875, -1.2875, -1.2875]]]
4
13
2


We can print the ks energies and weight for each kpoint

In [7]:
for k,e,w in zip(results.kpoints,results.evals,results.weights):
    print(k,e,w)

[0.0, 0.0, 0.0] [-0.21133344  0.22609091  0.22609158  0.22609158] [0.03125]
[-0.25, 0.25, -0.25] [-0.18214741  0.08223971  0.19833617  0.19833665] [0.1875]
[0.5, -0.5, 0.5] [-0.12643697 -0.02857772  0.18225845  0.18225893] [0.09375]
[0.0, 0.5, 0.0] [-0.17184867  0.09655427  0.15718577  0.15718597] [0.1875]
[0.75, -0.25, 0.75] [-0.11225404 -0.00768499  0.09723695  0.14564839] [0.375]
[0.5, 0.0, 0.5] [-0.13680575  0.02516734  0.08941682  0.17760265] [0.1875]
[0.0, -1.0, 0.0] [-0.0601889  -0.06018847  0.12203533  0.1220355 ] [0.09375]
[-0.5, -1.0, 0.0] [-0.05432256 -0.0543221   0.08587568  0.08587588] [0.1875]
[-0.25, 0.25, 0.25] [-0.18214737  0.0822393   0.19833657  0.19833657] [0.0625]
[0.5, -0.5, -0.5] [-0.12643666 -0.02857835  0.18225881  0.18225881] [0.03125]
[-0.75, 0.75, -0.25] [-0.11225406 -0.00768489  0.09723659  0.1456486 ] [0.1875]
[0.75, -0.75, -0.25] [-0.11225386 -0.00768547  0.09723703  0.14564857] [0.1875]
[-0.5, 0.5, 0.0] [-0.13680565  0.02516691  0.08941689  0.17760293] [

There are also several get methods, for instance

In [8]:
results.get_fermi()

Fermi energy attribute not found in the ouput file. Maybe `fixed` occupation type is used?


The method get_evals  return an array with the ks energies for each kpoint. The energies are expressed in eV and the 
energy of VBM is used as reference. A gap, both direct or indirect, can be set. In this case the energies of the empty ks states is
shifted. This procedure __does not__ update the occupation levels of the empty states

In [15]:
results.get_evals(set_gap=1.2)[0]

There are no empty bannds. `Set gap` has not been applied


array([-1.19029411e+01, -1.84464681e-05, -5.08766362e-10,  0.00000000e+00])

In [16]:
results.get_gap()

There are no empty states. Gap cannot be computed.


We test the PwParser using a nscf output file, in this case empty bands are present and the gap can be computed

In [17]:
result_nscf = P.PwParser('QeCalculator_test/si_nscf_test1.save/data-file-schema.xml')

Parse file : QeCalculator_test/si_nscf_test1.save/data-file-schema.xml


In [18]:
result_nscf.occupations[0]

array([1., 1., 1., 1., 0., 0., 0., 0.])

In [19]:
result_nscf.get_gap()

Indirect gap system
Gap : 0.723410090028532 eV
Direct gap : 2.551225478562494 eV


{'gap': 0.723410090028532,
 'direct_gap': 2.551225478562494,
 'position_cbm': 6,
 'positon_vbm': 0}

In [21]:
result_nscf.get_evals(set_direct_gap=3.0)[0]

Apply a scissor of 0.44877452143750585 eV


array([-1.19029066e+01, -1.15202563e-05, -1.15202537e-05,  0.00000000e+00,
        3.00000000e+00,  3.00000485e+00,  3.00000485e+00,  3.60084671e+00])

we see that in this case the energy of the 5-th states has been shift to 3 eV to implement the set_direct_gap requirement. 

## Parse of a Graphene output file

__Run the Analysis_BandStructure notebook to produce the xml file used by the PwParser.__

We parse a graphene output file to test if systems without a gap are correctly parsed.

In [38]:
results_gra = P.PwParser('IO_files/graphene_scf.save/data-file-schema.xml')

Parse file : IO_files/graphene_scf.save/data-file-schema.xml


In [39]:
results_gra.occupations_kind

'smearing'

In [43]:
results_gra.occupations[0]

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       4.01725605e-29, 6.30572455e-37, 9.04884912e-42, 8.90138577e-58])

In [44]:
results_gra.get_gap()

Direct gap system
Gap : 3.110445328033673e-10 eV


{'gap': 3.110445328033673e-10,
 'direct_gap': 3.110445328033673e-10,
 'position_cbm': 18,
 'positon_vbm': 18}

In [45]:
results_gra.get_fermi()

Fermi energy attribute not found in the ouput file. Maybe `fixed` occupation type is used?
